In [1]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
df = pd.read_csv("/content/drive/My Drive/Kaggle Club/SARCASM PROJECT '25/train-balanced-sarcasm.csv").fillna(' ')
df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...
...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,..."
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...
1010823,1,Perhaps this is an atheist conspiracy to make ...,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...
1010824,1,The Slavs got their own country - it is called...,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...


#1. Data Cleaning & Normalization
- Convert text to lowercase → Ensures consistency in tokenization (important for LSTM, less so for BERT/RoBERTa).
- Expand contractions → Convert "can't" to "cannot" for better tokenization.
- Remove URLs, special characters, and emojis → Ensures cleaner text input.
- Remove HTML tags (if applicable).
- Remove extra whitespaces and line breaks.

In [5]:
!pip install contractions


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 12.9 MB/s eta 0:00:00


In [31]:
slang_dict = {
    "gtfo": "get the fuck out",
    "idk": "i don't know",
    "idfk": "i don't fucking know",
    "tbh": "to be honest",
    "smh": "shaking my head",
    "lmk": "let me know",
    "brb": "be right back",
    "btw": "by the way",
    "imo": "in my opinion",
    "imho": "in my humble opinion",
    "stfu": "shut the fuck up",
    "kys": "kill yourself",
    "wth": "what the hell",
    "wtf": "what the fuck",
    "fyi": "for your information",
    "ikr": "i know right",
    "nvm": "never mind",
    "gg": "good game",
    "glhf": "good luck have fun",
    "afk": "away from keyboard",
    "rofl": "rolling on the floor laughing",
    "lmao": "laughing my ass off",
    "lmfao": "laughing my fucking ass off",
    "lol": "laugh out loud",
    "omg": "oh my god",
    "omfg": "oh my fucking god",
    "thx": "thanks",
    "ty": "thank you",
    "tysm": "thank you so much",
    "np": "no problem",
    "ggwp": "good game well played",
    "hmu": "hit me up",
    "dm": "direct message",
    "irl": "in real life",
    "asap": "as soon as possible",
    "jk": "just kidding",
    "wyd": "what are you doing",
    "wbu": "what about you",
    "hbu": "how about you",
    "g2g": "got to go",
    "gtg": "got to go",
    "tgif": "thank god it's friday",
    "rn": "right now",
    "u": "you",
    "ur": "your",
    "rly": "really",
    "gonna": "going to",
    "wanna": "want to",
    "lemme": "let me",
    "dunno": "do not know",
    "gimme": "give me",
    "cuz": "because"

}

def expand_slang(text):
    words = text.split()
    return " ".join([slang_dict[word] if word in slang_dict else word for word in words])

In [32]:
import pandas as pd
import re
import contractions

def preprocessing_text(s):
    if not isinstance(s, str):
        return ""

    # Convert to lowercase and strip spaces
    s = s.lower().strip()

    # Remove multiple spaces
    s = re.sub(r'\s+', ' ', s).strip()

    # Remove newlines
    s = re.sub(r'\n', ' ', s)

    # Remove hashtags but keep the words (e.g., "#TheDreamBoys" → "TheDreamBoys")
    s = re.sub(r'#(\w+)', r'\1', s)

    # Expand contractions
    s = contractions.fix(s)

    # Remove URLs
    s = re.sub(r'http\S+|www\S+|https\S+', '', s)

    # Remove HTML tags
    s = re.sub(r'<.*?>', '', s)

    # Remove "r/" from subreddit mentions (e.g., "r/politics" → "politics")
    s = re.sub(r'\br/(\w+)', r'\1', s)

    # Space out punctuation
    s = re.sub(r"([?!,+=—&%'\";:¿।।।|\(\){}\[\]//])", r" \1 ", s)

    # expand slang
    s = expand_slang(s)

    # Remove emojis
    emoji_pattern = re.compile("["
      u"\U0001F000-\U0001F9FF"  # Most emojis and symbols
      u"\U00002702-\U000027B0"  # Dingbats
      u"\U000024C2-\U0001F251"  # Enclosed characters
      u"\U0001F926-\U0001F937"  # Extra emoticons
      u"\U0001F1E0-\U0001F1FF"  # Flags
      u"\U00002600-\U000026FF"  # Miscellaneous Symbols
      u"\U00002700-\U0000277F"  # More Dingbats
      u"\U00002B50"             # Star
      u"\U00002640-\U00002642"  # Gender symbols
      u"\U00002600-\U00002B55"  # Other symbols
      u"\U00002B05-\U00002B07"  # Arrows
      u"\U00002934-\U00002935"  # Other symbols
      u"\U00003030"             # Wavy dash
      u"\U0000303D"             # Part Alternation Mark
      u"\U0001F170-\U0001F171"  # Letter symbols
      u"\U0001F17E-\U0001F17F"  # More letter symbols
      u"\U0001F18E"             # AB button
      u"\U0001F191-\U0001F19A"  # Other symbols
      u"\U0001F1E6-\U0001F1FF"  # Regional indicators (flags)
      u"\U0001F201-\U0001F202"  # Squared katakana
      u"\U0001F21A"             # Other squared symbols
      u"\U0001F22F"             # Squared CJK symbols
      u"\U0001F232-\U0001F23A"  # More CJK squared symbols
      u"\U0001F250-\U0001F251"  # Other squared symbols
      "]+", flags=re.UNICODE)

    s = emoji_pattern.sub(r'', s)

    # Remove non-ASCII characters
    s = re.sub(r'[^\x00-\x7F]+', '', s)


    return s


# Apply preprocessing
df['cleaned_comment'] = df['comment'].astype(str).apply(preprocessing_text)



print("Preprocessing complete. Saved as cleaned_reddit_comments.csv")


Preprocessing complete. Saved as cleaned_reddit_comments.csv


In [33]:
# side-by-side comparison
df.drop(columns=['author','subreddit','score','ups','downs','date','created_utc','parent_comment'], inplace=True)
df

,label,comment,cleaned_comment
0,0,NC and NH.,nc and nh.
1,0,You do know west teams play against west teams...,you do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G...","they were underdogs earlier today , but since ..."
3,0,"This meme isn't funny none of the ""new york ni...","this meme is not funny none of the "" new york ..."
4,0,I could use one of those tools.,i could use one of those tools.
...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,i am sure that iran and n. korea have the tech...
1010822,1,"whatever you do, don't vote green!","whatever you do , do not vote green !"
1010823,1,Perhaps this is an atheist conspiracy to make ...,perhaps this is an atheist conspiracy to make ...
1010824,1,The Slavs got their own country - it is called...,the slavs got their own country - it is called...


In [34]:
# save just the cleaned comment
df['comment'] = df['cleaned_comment']
df.drop(columns=['cleaned_comment'], inplace=True)
df

,label,comment
0,0,nc and nh.
1,0,you do know west teams play against west teams...
2,0,"they were underdogs earlier today , but since ..."
3,0,"this meme is not funny none of the "" new york ..."
4,0,i could use one of those tools.
...,...,...
1010821,1,i am sure that iran and n. korea have the tech...
1010822,1,"whatever you do , do not vote green !"
1010823,1,perhaps this is an atheist conspiracy to make ...
1010824,1,the slavs got their own country - it is called...


In [35]:
output_path = "/content/drive/My Drive/Kaggle Club/SARCASM PROJECT '25/cleaned_reddit_comments.csv"
df.to_csv(output_path, index=False)